In [3]:
!pip list


Package                   Version
------------------------- --------------
accelerate                0.26.0
aiohappyeyeballs          2.4.3
aiohttp                   3.10.10
aiosignal                 1.3.1
anyio                     4.6.2.post1
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     24.2.0
babel                     2.16.0
beautifulsoup4            4.12.3
bleach                    6.1.0
certifi                   2024.8.30
cffi                      1.17.1
charset-normalizer        3.4.0
comm                      0.2.2
datasets                  3.0.1
debugpy                   1.8.7
decorator                 5.1.1
defusedxml                0.7.1
dill                      0.3.8
executing                 2.1.0
fastjsonschema            2.20.0
filelock                  3.16.1
fqdn                      1.5.1
frozenlist                1.4.1
fsspec  

In [5]:
import torch

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
device

'cuda'

In [6]:

import numpy

In [7]:
!pip install datasets

In [8]:
import argparse
import logging
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer,DataCollatorForLanguageModeling

from datasets import load_dataset

/home/fossi/Documents/email_generation/myenvy/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
ds = load_dataset("Yale-LILY/aeslc")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [10]:
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-medium")

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 14436
    })
    validation: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 1960
    })
    test: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 1906
    })
})

In [13]:
def tokenize_function(examples):
  return tokenizer(examples["email_body"], examples['subject_line'], truncation=True,max_length=128,padding=True)

In [14]:
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [15]:
ds

DatasetDict({
    train: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 14436
    })
    validation: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 1960
    })
    test: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 1906
    })
})

In [16]:
tokenized_dataset = ds.map(tokenize_function, batched=True)

Map: 100%|████████████████████████| 1960/1960 [00:00<00:00, 11648.53 examples/s]


In [17]:
# from datasets import DatasetDict
# import torch

# # Assuming `dataset_dict` is your DatasetDict
# dataset_dict = DatasetDict({
#     'train': ... ,  # your train dataset
#     'validation': ... ,  # your validation dataset
#     'test': ...  # your test dataset
# })

# # Function to convert dataset to tensors
# def convert_to_tensor(dataset):
#     return {
#         'input_ids': torch.tensor(dataset['input_ids']),
#         'attention_mask': torch.tensor(dataset['attention_mask']),
#         'email_body': dataset['email_body'],  # Keep as is if you want to process text later
#         'subject_line': dataset['subject_line'],  # Keep as is if you want to process text later
#     }

In [18]:


# # Convert each split to tensors
# tensor_datasets = {
#     split: convert_to_tensor(dataset) for split, dataset in dataset_dict.items()
# # }

In [19]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [20]:
training_args = TrainingArguments(
    output_dir="./email_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    learning_rate=0.001,
    save_steps=1000,
    save_total_limit=2,
    fp16=True,  # Enable mixed precision training
    # no_cuda=False,  # No need for this if using 'device'
    # **device=device,**  # Pass the device to TrainingArguments
)

In [2]:
# !pip install transformers[torch]

In [22]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    data_collator=data_collator,
)


In [24]:
trainer

In [25]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [26]:
trainer.save_model("./email_model")